# Web Scraping Jobstreet

In [ ]:
#!pip install selenium

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
browser = webdriver.Chrome("chromedriver.exe")
browser.get("https://myjobstreet.jobstreet.co.id/home/login.php?site=id")

In [ ]:
username = browser.find_element(By.ID, "login_id")
username.send_keys("masukkan email")

password = browser.find_element(By.ID, "password")
password.send_keys("masukkan password")

time.sleep(2)

login_button = browser.find_element(By.XPATH, "//button[@name = 'btn_login']")
login_button.click()

### Scrap Multiple Page -> Untuk Search Term List (Beberapa Kata Kunci)

In [ ]:
# FUNCTION UNTUK MENCARI DARI SEARCH TERM LIST

def job_jobstreet_list(search_term_list, search_loc):
    
    # Data frame awal
    df = pd.DataFrame(columns = ["job_name", "company", "lokasi", 
                                 "Tingkat Pekerjaan", "Kualifikasi", "Pengalaman Kerja", "Jenis Pekerjaan", "Spesialisasi Pekerjaan", 
                                 "No. Registrasi", "Ukuran Perusahaan", "Waktu Proses Lamaran", "Industri", "Tunjangan dan Lain-lain", "Lokasi", 
                                 "gaji"])
    
    # For looping untuk "search term list"
    for search_term in search_term_list:
    
        # Isian untuk cari berdasarkan pekerjaan dan lokasi
        search_term_isi = browser.find_element(By.ID, "searchKeywordsField")
        search_term_isi.send_keys(search_term)

        search_loc_isi = browser.find_element(By.ID, "locationAutoSuggest")
        search_loc_isi.send_keys(search_loc)

        time.sleep(1)

        search_button = browser.find_element(By.XPATH, "//button[@type = 'submit']")
        search_button.click()

        browser.implicitly_wait(10)

        #-------------------------------------------------------------------------------
        # UNTUK PAGE = 1 (langsung scraping saja karena tidak perlu klik page button-nya)
        # Scroll sampai bawah dan kembalikan ke atas
        element = browser.find_element(By.TAG_NAME, "body")
        for i in range(1, 15):
            browser.execute_script("window.scrollBy(0, 560)", "")
            time.sleep(0.5)
        time.sleep(1)
        element.send_keys(Keys.HOME)

        # Melihat banyaknya pages
        pagination = browser.find_elements(By.XPATH, "//select[@id = 'pagination']/option")
        max_pages = int(pagination[-1].text)
        pages = [x for x in range(2, max_pages+1)]

        # List nama-nama lowongan
        job_all = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq8bm']/h1/a")
        list_job_oke = [i for i in job_all if search_term in i.text.lower()]

        for job in list_job_oke:
            job.click()
            time.sleep(1)

            # 1. BAGIAN AWAL
            list_hal_awal1 = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86m']")
            list_hal_awal2 = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86a']")

            list_hapus1 = []
            for i in range(0, len(list_hal_awal1)):
                if list_hal_awal1[i].text == "":
                    list_hapus1.append(i)
            del list_hal_awal1[list_hapus1[0]:list_hapus1[-1]+1]

            if len(list_hal_awal1) == 2:
                job_name = list_hal_awal1[0].text
                company = list_hal_awal1[1].text
            elif len(list_hal_awal1) == 3:
                job_name = list_hal_awal1[0].text + list_hal_awal1[1].text
                company = list_hal_awal1[2].text

            if len(list_hal_awal2) == 2:
                lokasi = list_hal_awal2[0].text
                gaji = np.nan
            elif len(list_hal_awal2) == 3:
                lokasi = list_hal_awal2[0].text
                gaji = list_hal_awal2[1].text

            # 2. BAGIAN INFORMASI TAMBAHAN
            info = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86i']/span")
            list_info = [i.text for i in info]

            del list_info[-1] # Index terakhir itu kosong ("")
            nama_kolom = list_info[0:len(list_info):2]

            del list_info[0:len(list_info):2] # Hapus kolomnya
            isi_kolom = list_info # Ini list_info yang sudah kehapus nama kolomnya
            dict_info = dict(zip(nama_kolom, isi_kolom))

            dict1 = {"job_name": job_name, "company": company, "lokasi": lokasi}
            dict1.update(dict_info)
            dict1.update({"gaji": gaji})

            df = df.append(dict1, ignore_index = True) 

        # Untuk ke page 2 -> tidak ada tombol "Sebelumnya"
        klik_next = browser.find_element(By.XPATH, "//a/span[@class = 'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _1d0g9qk4 _18qlyvc5']")
        klik_next.click()
        #time.sleep(2)
        browser.implicitly_wait(10)

        #-------------------------------------------------------------------------------
        # UNTUK PAGE > 1 (perlu klik page button-nya)

        # For loop untuk page
        for page in pages:  

            # Scroll sampai bawah dan kembalikan ke atas
            element = browser.find_element(By.TAG_NAME, "body")
            for i in range(1, 15):
                browser.execute_script("window.scrollBy(0, 560)", "")
                time.sleep(0.5)
            time.sleep(1)
            element.send_keys(Keys.HOME)

            # List nama-nama lowongan
            job_all = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq8bm']/h1/a")
            list_job_oke = [i for i in job_all if search_term in i.text.lower()]

            for job in list_job_oke:
                job.click()
                time.sleep(1)

                # 1. BAGIAN AWAL
                list_hal_awal1 = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86m']")
                list_hal_awal2 = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86a']")

                list_hapus1 = []
                for i in range(0, len(list_hal_awal1)):
                    if list_hal_awal1[i].text == "":
                        list_hapus1.append(i)
                del list_hal_awal1[list_hapus1[0]:list_hapus1[-1]+1]

                if len(list_hal_awal1) == 2:
                    job_name = list_hal_awal1[0].text
                    company = list_hal_awal1[1].text
                elif len(list_hal_awal1) == 3:
                    job_name = list_hal_awal1[0].text + list_hal_awal1[1].text
                    company = list_hal_awal1[2].text

                if len(list_hal_awal2) == 2:
                    lokasi = list_hal_awal2[0].text
                    gaji = np.nan
                elif len(list_hal_awal2) == 3:
                    lokasi = list_hal_awal2[0].text
                    gaji = list_hal_awal2[1].text

                # 2. BAGIAN INFORMASI TAMBAHAN
                info = browser.find_elements(By.XPATH, "//div[@class = 'sx2jih0 zcydq86i']/span")
                list_info = [i.text for i in info]

                del list_info[-1] # Index terakhir itu kosong ("")
                nama_kolom = list_info[0:len(list_info):2]

                del list_info[0:len(list_info):2] # Hapus kolomnya
                isi_kolom = list_info # Ini list_info yang sudah kehapus nama kolomnya
                dict_info = dict(zip(nama_kolom, isi_kolom))

                dict1 = {"job_name": job_name, "company": company, "lokasi": lokasi}
                dict1.update(dict_info)
                dict1.update({"gaji": gaji})

                df = df.append(dict1, ignore_index = True)

            # Untuk ke page >2 -> ada tombol "Sebelumnya"
            klik_next = browser.find_elements(By.XPATH, "//a/span[@class = 'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _1d0g9qk4 _18qlyvc5']")

            if len(klik_next) == 2:
                klik_next[-1].click()
                #time.sleep(2)
                browser.implicitly_wait(10)
                
        browser.implicitly_wait(5)
        search_term_isi.send_keys(Keys.CONTROL + "a")
        search_term_isi.send_keys(Keys.DELETE)
        search_loc_isi.send_keys(Keys.CONTROL + "a")
        search_loc_isi.send_keys(Keys.DELETE)

    return df

In [ ]:
start_time = datetime.now()

search_term_list = ["data scientist", "data analyst", "data engineer"]
hasil = job_jobstreet_list(search_term_list, "indonesia")

end_time = datetime.now()
print("Durasi Scraping: {}".format(end_time - start_time))

In [ ]:
len(hasil)

In [ ]:
# Save to csv
hasil.to_csv("Scraping Profesi Data (Jobstreet).csv", index = False)

# Data diambil tanggal 6 Des 2021

In [ ]:
browser.close()

In [ ]:
# Membaca data yang sudah disimpan
df = pd.read_csv("Scraping Profesi Data (Jobstreet).csv")
df.tail()